https://towardsdatascience.com/image-captioning-using-deep-learning-fe0d929cf337




In [ ]:
!pip install opencv-python
!apt-get update && apt-get install libgl1 -y

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import cv2
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, Conv2D, Concatenate, Flatten, Add, Dropout, GRU
import random
import datetime
from nltk.translate.bleu_score import sentence_bleu

In [2]:
def get_random_set():
    obj = reqd[np.random.randint(len(reqd))]
    for i in range(len(obj["imgs"])):
        plt.subplot(1, len(obj["imgs"]),i+1)
        plt.axis('off')
        img = plt.imread(get_img_path(obj["imgs"][i]))
        plt.imshow(img)
    print(obj["desc"])

def get_random_set2(df, k):
    obj = df[k]
    for i in range(len(obj["imgs"])):
        plt.subplot(1, len(obj["imgs"]),i+1)
        plt.axis('off')
        img = plt.imread(get_img_path(obj["imgs"][i]))
        plt.imshow(img)
    print(obj["desc"])
    
def get_img_path(x):
    return "./imgs/"+x+".png"

In [3]:
train_dataset = pd.read_csv('df_train.csv')
test_dataset = pd.read_csv('df_test.csv')
cv_dataset = pd.read_csv('df_cv.csv')

In [4]:
train_dataset

,Unnamed: 0,desc,img1,img2,img3,img4,img5
0,0,startseq right upper lobe airspace disease con...,CXR3069_IM-1432-1001,CXR3069_IM-1432-2001,NaN,NaN,NaN
1,1,startseq mediastinal contours are normal lungs...,CXR2629_IM-1116-1001,CXR2629_IM-1116-2001,NaN,NaN,NaN
2,2,startseq cardiomediastinal contours are unchan...,CXR1888_IM-0576-1001,CXR1888_IM-0576-4004,NaN,NaN,NaN
3,3,startseq heart size normal the lungs are clear...,CXR2248_IM-0844-1001,CXR2248_IM-0844-1002,NaN,NaN,NaN
4,4,startseq the cardiomediastinal contours are wi...,CXR3408_IM-1648-1001,CXR3408_IM-1648-1002,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2063,2331,startseq the heart normal size the mediastinum...,CXR2243_IM-0840-1001,CXR2243_IM-0840-2001,CXR2243_IM-0840-3001,CXR2243_IM-0840-4001,NaN
2064,2332,startseq normal heart size normal mediastinal ...,CXR1356_IM-0231-1001,CXR1356_IM-0231-2001,NaN,NaN,NaN
2065,2333,startseq frontal and lateral views the chest s...,CXR1883_IM-0572-1001,CXR1883_IM-0572-2001,NaN,NaN,NaN
2066,2334,startseq normal heart size and mediastinal con...,CXR2622_IM-1110-1001,CXR2622_IM-1110-1002,NaN,NaN,NaN


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [6]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(train_dataset.desc)

In [8]:
vocab_size = len(tokenizer.word_counts) + 1
vocab_size

1313

In [9]:
# import gensim
from gensim.scripts import glove2word2vec

In [10]:
try:
    with open(r"embedding_matrix.pickle", "rb") as output_file:
        embedding_matrix = pickle.load(output_file)
    print("loaded")
except:
    embedding_matrix = np.zeros((vocab_size,300))
    filepath = r"./utils/glove.6B/glove.6B.300d.txt"
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in tokenizer.word_index.keys():
                idx = tokenizer.word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:300]
    with open(r"embedding_matrix.pickle", "wb") as output_file:
        pickle.dump(embedding_matrix, output_file)

loaded


In [11]:
from tensorflow.keras.applications import densenet
chex = densenet.DenseNet121(include_top=False, weights = None,   input_shape=(224,224,3), pooling="avg")
X = chex.output
X = Dense(14, activation="sigmoid", name="predictions")(X)
model = Model(inputs=chex.input, outputs=X)
model.load_weights(r'./utils/Image_features_enc.h5')
chexnet = Model(inputs = model.input, outputs = model.layers[-2].output)

In [ ]:
chexnet.predict(load_image(get_img_path(train_dataset.values[0][2])))

In [ ]:
chexnet.summary()

In [13]:
from PIL import Image
def load_image(img_name):
    '''Function to load the image'''
    image = Image.open(img_name)
    image = image.resize((224,224))
    image_array = np.asarray(image.convert("RGB"))
    image_array = image_array / 255.
    
    X = np.expand_dims(image_array, axis=0)
    X = np.asarray(X) 
    return X

In [ ]:
Xnet_features = {}
for item in train_dataset.values:
# item = train_dataset.values[0]
img1 = load_image(get_img_path(item[2]))
img1_features = chexnet.predict(img1)    
# img2 = load_image(get_img_path(item[3]))
# img2_features = chexnet.predict(img2)
# input_ = np.concatenate((img1_features, img2_features), axis=1)
print(img1_features)
# break
#     Xnet_features[key] = input_

In [ ]:
len(train_dataset.values)